In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
project_dir = 'C:\\Users\\PLDD\\python\\Python\\ML\\Kaggle\\house_price'
raw_path = os.path.join(project_dir,'data','raw')
train_path = os.path.join(raw_path, 'train.csv')
test_path = os.path.join(raw_path, 'test.csv')

 # Create data frame

Combines train and test data frame:

In [2]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
test_df['SalePrice'] = 0.0

Define and fill some nan value with simple NoNe that means - this property attribute is absent

In [4]:
DefSeqCat = ['Alley','BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature']
UndefSeqCat1 = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd','Electrical', 'KitchenQual',
            'Functional', 'SaleType'] # there are defenetly lost elements 
UndefSeqCat2 = ['MasVnrType'] # i may just not make it clear the property of the column
UndefSeqNum = ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 
               'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

Replace NAN values in 'LotFrontage' with iterpolated value.

In [8]:
df = train_df.append(test_df)
df.index = list(range(train_df.index.size + test_df.index.size))

yearlable = ['1906', '1942', '1975', '1991', '2011']
yearbin = [1800,1906,1942,1975, 1991, 2011]
df['YearInt']=pd.cut(x = df['YearBuilt'].values, bins = yearbin, right = False, labels = yearlable)

replace=dict.fromkeys(DefSeqCat, 'NoNe')
df = df.fillna(replace)

In [10]:
#case 1 - try to fill 'LotFrontage' with mean 'YearInt' data:
catclass = ['Neighborhood', 'LotConfig','LotShape','YearInt']
fullclass = catclass + ['LotFrontage']
validdf = df[fullclass].dropna(axis = 0, how = 'any')
nandf = df[df['LotFrontage'].isnull()]
pvt = validdf.pivot_table(values = 'LotFrontage', index = catclass, aggfunc = np.mean)
pvt=pvt[pvt.notnull().all(1)]
t1 = pvt.loc[[tuple(x) for x in nandf[catclass].values]]
t1.index = nandf.index
df.loc[nandf.index,'LotFrontage'] = t1['LotFrontage']
print(df[df['LotFrontage'].isnull()].index.size)

#case 2
validdf = df[fullclass].dropna(axis = 0, how = 'any')
#nandf = df[df['LotFrontage'].isnull()]
#pvt = validdf.pivot_table(values = 'LotFrontage', index = catclass, aggfunc = np.mean)
#pvt=pvt[pvt.notnull().all(1)]
#t1 = pvt.loc[[tuple(x) for x in nandf[catclass].values]]
#t1.index = nandf.index
#df.loc[nandf.index,'LotFrontage'] = t1['LotFrontage']


81


In [382]:
#import matplotlib.pyplot as plt
#plt.figure()
#nandf = df[df['LotFrontage'].isnull()]
#nandf['YearBuilt'].hist(bins = 100)
#plt.show()
#yy
#plt.plot()
#nandf.pivot_table(values = 'LotFrontage', index = ['YearBuilt'], aggfunc = np.size).sort_values(by='LotFrontage', axis = 0)
#tt=nandf.pivot_table(values = 'LotFrontage', index = catclass, aggfunc = np.size)
#tt[tt['LotFrontage'].notnull()].sort_values(by='LotFrontage', axis = 0)
#tt.sort_values(by='LotFrontage', axis = 0)



In [384]:
yy.columns

MultiIndex(levels=[['std', 'mean', 'size', 'amin', 'amax', 'RelStd'], ['LotFrontage']],
           labels=[[0, 1, 2, 3, 4, 5], [0, 0, 0, 0, 0, 0]])

In [385]:
yy.index

MultiIndex(levels=[['CollgCr', 'Gilbert', 'NAmes', 'NWAmes', 'Sawyer'], ['Corner', 'CulDSac', 'FR2', 'FR3', 'Inside'], ['IR1', 'IR2', 'IR3', 'Reg'], ['1906', '1942', '1975', '1991', '2011']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4], [0, 0, 0, 0, 1, 1, 1, 2, 2, 4, 4, 4, 4, 4, 4, 4, 0, 1, 2, 4, 4, 4, 4, 4, 0, 0, 0, 1, 2, 2, 4, 4, 4, 4, 0, 0, 0, 0, 1, 4, 4, 4, 4, 4, 0, 0, 1, 2, 4, 4, 4, 4], [0, 3, 3, 3, 0, 0, 1, 1, 3, 0, 0, 0, 1, 3, 3, 3, 0, 0, 0, 0, 1, 3, 3, 3, 0, 3, 3, 0, 1, 3, 0, 3, 3, 3, 0, 0, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3], [4, 2, 3, 4, 3, 4, 4, 4, 4, 2, 3, 4, 4, 2, 3, 4, 4, 4, 4, 4, 4, 2, 3, 4, 2, 1, 2, 2, 4, 2, 2, 1, 2, 4, 2, 3, 2, 3, 3, 2, 3, 2, 3, 4, 1, 2, 2, 2, 2, 1, 2, 3]],
           names=['Neighborhood', 'LotConfig', 'LotShape', 'YearInt'])